In [1]:
# importing all the packages I'll need
import os
import pandas as pd
import numpy as np

# Wrangling Covid-19 Vaccination Rate from Our World in Data and Income Classification from World Bank
Raw Data:
- [OWID COVID-19 vaccine doses administered per 100 people](https://ourworldindata.org/grapher/covid-vaccination-doses-per-capita?tab=map&country=Europe~Africa~North+America~South+America~Asia~Oceania~Low+income~High+income~Lower+middle+income~Upper+middle+income)
- [World Bank Income Classification](https://datahelpdesk.worldbank.org/knowledgebase/articles/906519-world-bank-country-and-lending-groups)

Objectives:
1. Load the Covid-19 vaccine rate from OWiD and World Bank income classification
2. Merge the two tables that will be used as the raw data of my analysis
3. Revamp the data to only show the monthly progress of each country's vaccination rate

### Load data

In [2]:
# Load OWiD vaccine rate data
df_vax = pd.read_csv('cvdpc.csv')

# Load World bank income classification data
df_class = pd.ExcelFile('CLASS.xlsx').parse('List of economies').dropna(how='all').copy().reset_index().drop(0)

### Checking the content of the data

In [3]:
def print_summary(title, value):
    print('-' * 80, title, '-' * 80, value, sep='\n')

def summarize_dataframe(frame):
    print_summary('Dataset Summary', frame.describe())
    print_summary('Null Values Count', frame.isnull().sum())

    for col in frame:
        print_summary(f'Unique values in column "{col}"', frame[col].unique())

In [4]:
summarize_dataframe(df_vax)

--------------------------------------------------------------------------------
Dataset Summary
--------------------------------------------------------------------------------
       total_vaccinations_per_hundred
count                    50660.000000
mean                        79.910846
std                         68.151054
min                          0.000000
25%                         14.927500
50%                         67.470000
75%                        132.770000
max                        352.440000
--------------------------------------------------------------------------------
Null Values Count
--------------------------------------------------------------------------------
Entity                               0
Code                              7439
Day                                  0
total_vaccinations_per_hundred       0
dtype: int64
--------------------------------------------------------------------------------
Unique values in column "Entity"
-----------------

In [5]:
summarize_dataframe(df_class)

--------------------------------------------------------------------------------
Dataset Summary
--------------------------------------------------------------------------------
            index
count  263.000000
mean   132.174905
std     76.316735
min      1.000000
25%     66.500000
50%    132.000000
75%    197.500000
max    264.000000
--------------------------------------------------------------------------------
Null Values Count
--------------------------------------------------------------------------------
index                    0
Economy                  0
Code                     0
Region                  46
Income group            47
Lending category       119
Other (EMU or HIPC)    205
dtype: int64
--------------------------------------------------------------------------------
Unique values in column "index"
--------------------------------------------------------------------------------
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  2

### Merging the two data

In [6]:
df_vax_class = df_vax.merge(df_class, on = "Code", how = "inner").drop(columns = ["Economy", "Lending category", "Other (EMU or HIPC)"])
df_vax_class.columns = ['Country', 'Code', 'Date', 'Vax_Rate', 'Index', 'Region', 'Income_group']

In [7]:
summarize_dataframe(df_vax_class)

--------------------------------------------------------------------------------
Dataset Summary
--------------------------------------------------------------------------------
           Vax_Rate         Index
count  41798.000000  41798.000000
mean      79.320283    103.594670
std       68.616715     60.177529
min        0.000000      1.000000
25%       14.812500     53.000000
50%       64.660000    100.000000
75%      131.907500    152.000000
max      352.440000    217.000000
--------------------------------------------------------------------------------
Null Values Count
--------------------------------------------------------------------------------
Country          0
Code             0
Date             0
Vax_Rate         0
Index            0
Region           0
Income_group    29
dtype: int64
--------------------------------------------------------------------------------
Unique values in column "Country"
---------------------------------------------------------------------------

## Group by Month

In [8]:
# obtain the country codes in the table
country_code = df_vax_class["Code"].unique()

In [9]:
# Convert the "Date" column to datetime format
df_vax_class['Date'] = pd.to_datetime(df_vax_class['Date'])

In [10]:
# create the empty data frame for the new monthly vaccination rate table
df_vax_month = pd.DataFrame()

For some reasons, if I just obtain the data for the last date of each month, the months that only contain 1 data are deleted. Therefore I re-grouped the data to obtain the last 2 dates of each month.

The code will loop each country's data and obtain the last two rows of each month.

In [11]:
for code in country_code:
    data = df_vax_class[df_vax_class["Code"].str.contains(code)]
    data = data.groupby(data["Date"].dt.month, group_keys="Code").tail(2)
    
    # Change the date time format to "yyyy=mm"
    data['Date'] = data['Date'].dt.strftime('%Y-%m') 
    df_vax_month = pd.concat([df_vax_month, data])

In [12]:
# Obtain all the year-month value in the table
vax_date = df_vax_month.Date.unique()

## Revamping the Data
After reinspecting the data, turns out our World in Data only show the reported covid-19 vaccination rate for each month, instead of continuously retaining the data. Therefore, I have to revamp the data so table still keeps the last data from previous month during the months the country does not report their data.

In [13]:
# New dataframe for the revamped data
df_cont_monthly_vax = pd.DataFrame()

Now, the code will loop the data from each country and recreate a table starting from the first month they reported their latest vaccination rate until April 2022.

In [14]:
for code in country_code:
    # New dataframe to retain the latest vaccination rate data of each country
    previous_mth = pd.DataFrame()
    
    # Each looped country dataframe
    current_row = df_vax_month[df_vax_month["Code"].str.contains(code)]
    
    for mth in vax_date:
        # The year-month of each looped "Date" dataframe
        current_mth = current_row[current_row["Date"].str.contains(mth)]
        
        # If current month contains empty data, then the current month is filled with previous month's data
        if current_mth["Vax_Rate"].empty:
            previous_mth["Date"] = mth
            df_cont_monthly_vax = pd.concat([df_cont_monthly_vax, previous_mth])
            continue
        
        # Otherwise, continue adding the data
        else:
            df_cont_monthly_vax = pd.concat([df_cont_monthly_vax, current_mth])
            
        previous_mth = current_mth.copy()

Since when the data was grouped by the last two data from each month, so there are duplicates for each month data in the table. Therefore, the duplicate needs to be eliminated by retaining only the last data from each month, then sort it by date for further analysis and visualization.

In [15]:
df_cont_monthly_vax.drop_duplicates(subset=['Code', 'Date'], keep='last', inplace = True, ignore_index = True)
df_cont_monthly_vax = df_cont_monthly_vax.sort_values(by='Date')

In [16]:
# Checking for the remaining missing data to be further cleaned

for code in country_code:
    current_row = df_cont_monthly_vax[df_cont_monthly_vax["Code"].str.contains(code)]
    current_country_date_list = [new_date for new_date in current_row["Date"]]
    for date in vax_date:
        current_row = current_row[current_row["Date"].str.contains(date)]
        
        if date not in current_country_date_list:
            if date > df_vax_month[df_vax_month["Code"].str.contains(code)].values[0][2]:
                print(code, date, df_vax_month[df_vax_month["Code"].str.contains(code)].values[0][2])


DZA 2021-04 2021-01
DZA 2021-05 2021-01
DZA 2021-06 2021-01
DZA 2021-07 2021-01
IDN 2021-04 2021-01
MUS 2021-04 2021-01
TKM 2021-04 2021-02
TKM 2021-05 2021-02
TKM 2021-06 2021-02
TKM 2021-07 2021-02


#### Saving the new revamped data to be further cleaned and analyzed

In [17]:
df_cont_monthly_vax.to_csv('Cov-Vax-Income-Monthly_Continuous.csv', sep = ",", index = False)